# Django Notes


## templatetags
template tags are usually cryptic statements written in python html pages. It gets compiled to become complete html.

> Create Template tags directory in python app.

```
polls/
    __init__.py
    models.py
    templatetags/
        __init__.py
        poll_extras.py
    views.py
    
```

And then in your template use the following:
```python
{% load poll_extras %}
```

This is a security feature, it tells which files to load without loading them all in your project. Gives more control to your app.

## Custom Filters

To be a valid template tag directory, the module must contain a module-level variable named register that is a template.Library instance, in which all tags and filters are registered.

```python
from django import template
register = template.Library()
```

```python
@register.filter(name=cut)
def cut(value, arg):
   return value.replace(arg)

@register.filter
def lower(value):
    return value.lower()
```

or we can manually register the template tags
```
register.filter('cut',cut)
register.filter('lower', lower)
```

# Custom template tags

tags are usually called with ** { % % } ** tags. they are usually complicated than filters and can take many number of arguments.

## Simple tags

django.template.Library.simple_tag()

Many template tags take a number of arguments - strigns or template variables - and return a string after doing some processing based solely on the input arguments and some external information. 

To ease the creation of these types of tags, Django provides a helper function, simple_tag. This function, which is a method of django.template.Library, takes a function that accepts any number of arguments, wraps it in a render function and the other necessary bits mentioned above and registers it with the template system

```python
import datetime
from django import template

register = template.Library()

@register.simple_tag
def current_time(format_string):
    return datetime.datetime.now().strftime(format_string)
```

If template tag needs to access current_context, you can use **takes_context** argument when registering your tag.

```python
@register.simple_tag(takes_context=True)
def current_time(context, format_string):
    timezone = context['timezone']
    return your_get_current_time_method(timezone, format_string)
```

Note that first argument must be called **context**

simple_tag also takes named arguments

```python
@register.simple_tag
def my_tag(a, b, *args, **kwargs):
    warning = kwargs['warning']
    profile = kwargs['profile']
    ...
    return ...
```

To use, we can simply call
> {% my_tag 123 "abcd" book.title warning=message %}

## Inclusion tag

Another common type of template tag is the type that displays some data by rendering another template.

```
{% show_results poll %}
```

should give the output as
```
<ul>
  <li> First choice </li>
  <li> Second choice </li>
  <li> third Choice </li>
</li>
```

```python
@register.inclusion_tag('results.html')
def show_results(poll):
    choices = poll.choice_set.all()
    return {'choices': choices}
```

Sometimes your template require more number of arguments than you can pass through function. For those scenarios, we can pass context parameter
```python
@register.inclusion_tag('link.html', takes_context=True)
def jump_link(context):
    return {
        'link': context['home_link'],
        'title': context['home_title'],
    }
    
    # with link.html as 
    Jump directly to <a href="{{ link }}">{{ title }}</a>
    
    # We can simply call then
    {% jump_link %}
```

## Assignment tag

This is used when tag result should be stored as another variable

```python
@register.assignment_tag
def get_current_time(format_string):
    return datetime.datetime.now().strftime(format_string)
    
# It can be used as:
{% get_current_time "%Y-%m-%d %I:%M %p" as the_time %}
<p>The time is {{ the_time }}.</p>
```

## Advanced custom tags